### **HuggingFace Login**

In [1]:
from huggingface_hub import login
login()

### **Import Libairies**

In [2]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
import sentencepiece
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from datasets import Dataset



### **Read Datasets**

In [3]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
'''

df_train = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv", index_col=0)
df_test = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv", index_col=0)


### **Short Analysis**

In [4]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


### **Preprocess Datasets**

##### _Merge columns (full)_

In [5]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
damage,41,9
body%20bags,41,9


In [6]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [7]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


### **Model Assessment**

In [8]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Value': [f1, precision, recall]
    }, index=['F1 Score', 'Precision', 'Recall'])
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

### **ALBERT Tokenizer**

In [9]:
albert_tokenizer_full = AlbertTokenizer.from_pretrained("albert-base-v2")

albert_encodings_full = albert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
albert_labels_full = torch.tensor(list(df_train_full['target']))

albert_input_ids_train_full, albert_input_ids_valid_full, albert_token_type_ids_train_full, albert_token_type_ids_valid_full, \
albert_attention_mask_train_full, albert_attention_mask_valid_full, albert_y_train_full, albert_y_valid_full = train_test_split(
    albert_encodings_full['input_ids'], 
    albert_encodings_full['token_type_ids'], 
    albert_encodings_full['attention_mask'], 
    albert_labels_full, 
    test_size=0.15, 
    stratify=albert_labels_full, 
    random_state=0
)

albert_train_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_train_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_train_full),
    'attention_mask': torch.tensor(albert_attention_mask_train_full)
}

albert_valid_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_valid_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(albert_attention_mask_valid_full)
}

albert_train_dataset_full = Dataset.from_dict({
    "input_ids": albert_train_encodings_full['input_ids'],
    "attention_mask": albert_train_encodings_full['attention_mask'],
    "labels": albert_y_train_full
})

albert_valid_dataset_full = Dataset.from_dict({
    "input_ids": albert_valid_encodings_full['input_ids'],
    "attention_mask": albert_valid_encodings_full['attention_mask'],
    "labels": albert_y_valid_full
})

albert_test_encodings_full = albert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)

albert_test_encodings_full = {
    key: torch.tensor(val) for key, val in albert_test_encodings_full.items()
}

albert_test_dataset_full = Dataset.from_dict({
    "input_ids": albert_test_encodings_full['input_ids'],
    "attention_mask": albert_test_encodings_full['attention_mask']
})

##### _Save HuggingFace ALBERT Tokenizer_

In [12]:
albert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_tokenizer")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--yanncauchepin--kaggle_disastertweets_albert_tokenizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_tokenizer/commit/7a7baa1173f3e18d03a3d765cd13fd98e9ee46d0', commit_message='Upload tokenizer', commit_description='', oid='7a7baa1173f3e18d03a3d765cd13fd98e9ee46d0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_albert_tokenizer'), pr_revision=None, pr_num=None)

### **ALBERT Training from scratch**

##### _ALBERT Transformers + local save_

In [18]:
albert_model_full = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_albert_model",
    output_dir="./albert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./albert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

albert_trainer_full = Trainer(
    model=albert_model_full,
    args=training_args,
    train_dataset=albert_train_dataset_full,
    eval_dataset=albert_valid_dataset_full
)

albert_trainer_full.train()

albert_trainer_full.save_model("albert_model") # Local

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/8090 [00:00<?, ?it/s]

{'loss': 0.793, 'grad_norm': 67.93590545654297, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.712, 'grad_norm': 44.848941802978516, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.6488, 'grad_norm': 60.00485610961914, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.7002, 'grad_norm': 49.69922637939453, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.6291, 'grad_norm': 20.778968811035156, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.5648, 'grad_norm': 22.72116470336914, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6075, 'grad_norm': 113.43247985839844, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.6722, 'grad_norm': 49.39283752441406, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6164, 'grad_norm': 44.678653717041016, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.6168, 'grad_norm': 41.77793502807617, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.6478, 'grad_norm': 47.632

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.6843461990356445, 'eval_runtime': 6.8235, 'eval_samples_per_second': 167.362, 'eval_steps_per_second': 41.914, 'epoch': 1.0}


KeyError: "The `metric_for_best_model` training argument is set to 'eval_f1', which is not found in the evaluation metrics. The available evaluation metrics are: ['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch']. Consider changing the `metric_for_best_model` via the TrainingArguments."

##### _Save HugggingFace trained ALBERT Transformers_ 

In [13]:
albert_trainer_full.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_model/commit/3a2b0e060519d1a32fc5bb3a50b9be67fd20c73c', commit_message='End of training', commit_description='', oid='3a2b0e060519d1a32fc5bb3a50b9be67fd20c73c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_albert_model'), pr_revision=None, pr_num=None)

### **ALBERT Loading from pretrained**

In [19]:
albert_model_full = AlbertForSequenceClassification.from_pretrained("albert_model")

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_albert_model",
    output_dir="./albert_results",
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./albert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

albert_trainer_full = Trainer(
    model=albert_model_full,
    args=training_args,
    train_dataset=albert_train_dataset_full,
    eval_dataset=albert_valid_dataset_full
)

### **ALBERT Assessment**

In [16]:
albert_predictions_full = albert_trainer_full.predict(albert_valid_dataset_full)
albert_logits_full = albert_predictions_full.predictions
albert_y_pred_full = np.argmax(albert_logits_full, axis=1)

albert_trainer_full_assessement = evaluate_classifier(albert_y_valid_full.numpy(), albert_y_pred_full)
display(albert_trainer_full_assessement[0])
display(albert_trainer_full_assessement[1])

  0%|          | 0/286 [00:00<?, ?it/s]

,Value
F1 Score,0.766772
Precision,0.783000
Recall,0.774081


,Predicted Negative,Predicted Positive
Actual Negative,589,62
Actual Positive,196,295


### **ALBERT Test Predictions**

##### _ALBERT Test Predictions_

In [ ]:
albert_test_predictions_full = albert_trainer_full.predict(albert_test_dataset_full)
albert_test_logits_full = albert_test_predictions_full.predictions
albert_test_y_pred_full = np.argmax(albert_test_logits_full, axis=1)

albert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': albert_test_y_pred_full.flatten()
})

##### _Save HuggingFace ALBERT Test Predictions_

In [ ]:
hf_albert_test_submission_full = Dataset.from_pandas(albert_test_submission_full)
hf_albert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_submission_df")